In [1]:
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter

# CONFIG
DATASET_PATH = "/kaggle/input/datasets/shahjahanabdullatif/annotated-dental-cavity-dataset/Yolo_Dataset"
ANALYSIS_OUT = "/kaggle/working/data_analysis"
os.makedirs(ANALYSIS_OUT, exist_ok=True)

def analyze_dataset():
    issue_records = []
    print("📊 Analyzing Dataset splits...")
    
    for split in ["train", "val", "test"]:
        img_path = os.path.join(DATASET_PATH, split, "images")
        lbl_path = os.path.join(DATASET_PATH, split, "labels")
        images = glob.glob(os.path.join(img_path, "*.*"))
        
        class_counter = Counter()
        for img in images:
            name = os.path.splitext(os.path.basename(img))[0]
            label = os.path.join(lbl_path, name + ".txt")
            
            if not os.path.exists(label):
                issue_records.append({"Split": split, "Issue": "Missing", "File": name})
                continue
            if os.path.getsize(label) == 0:
                issue_records.append({"Split": split, "Issue": "Empty", "File": name})
                continue
                
            with open(label, "r") as f:
                for line in f:
                    class_counter[int(line.strip().split()[0])] += 1
        
        print(f"✅ {split.upper()}: {len(images)} images | Distribution: {dict(class_counter)}")

    pd.DataFrame(issue_records).to_csv(f"{ANALYSIS_OUT}/label_issues.csv", index=False)
    print(f"✅ Audit report saved to {ANALYSIS_OUT}")

analyze_dataset()

📊 Analyzing Dataset splits...
✅ TRAIN: 23666 images | Distribution: {1: 94949, 0: 51557}
✅ VAL: 5071 images | Distribution: {1: 20754, 0: 10074}
✅ TEST: 5071 images | Distribution: {1: 25132, 0: 7804}
✅ Audit report saved to /kaggle/working/data_analysis


In [2]:
pip install -U ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 21.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import yaml
import torch
from ultralytics import YOLO

# CONFIG
SETTINGS = {
    "model_type": "yolov8m.pt",
    "epochs": 80,
    "imgsz": 640,
    "batch": 16, 
    "project": "Dental_Cavity_Research",
    "name": "v8m_full_research",
    "optimizer": "AdamW",
    "patience": 30,
}

OUTPUT_DIR = "/kaggle/working/final_results"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# 1. Create YAML
data_yaml = {
    'path': DATASET_PATH,
    'train': 'train/images', 'val': 'val/images', 'test': 'test/images',
    'nc': 2, 'names': ['Cavity', 'Non-Cavity']
}
with open(os.path.join(OUTPUT_DIR, "dataset.yaml"), 'w') as f:
    yaml.dump(data_yaml, f)

# 2. Train
def start_training():
    # Multi-GPU check
    device = [0, 1] if torch.cuda.device_count() > 1 else 0
    print(f"🚀 Training starting on: {device}")
    
    model = YOLO(SETTINGS["model_type"])
    
    model.train(
        data=os.path.join(OUTPUT_DIR, "dataset.yaml"),
        epochs=SETTINGS["epochs"],
        imgsz=SETTINGS["imgsz"],
        batch=SETTINGS["batch"],
        project=SETTINGS["project"],
        name=SETTINGS["name"],
        device=device,
        patience=SETTINGS["patience"],
        optimizer=SETTINGS["optimizer"],
        cos_lr=True,
        amp=True
    )

start_training()

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
🚀 Training starting on: [0, 1]
Ultralytics 8.4.14 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 14913MiB)
                                                      CUDA:1 (Tesla T4, 14913MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/kaggle/working/final_results/dataset.yaml, degrees=0.0, deterministic=True, device=0,1, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=80, erasing=0.4, exist_ok=False, fliplr=0

In [4]:
import zipfile

ZIP_NAME = "Dental_Research_Export.zip"
dirs_to_zip = ["/kaggle/working/Dental_Cavity_Research", "/kaggle/working/final_results", "/kaggle/working/data_analysis"]

with zipfile.ZipFile(ZIP_NAME, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for folder in dirs_to_zip:
        if os.path.exists(folder):
            for root, _, files in os.walk(folder):
                for file in files:
                    full_path = os.path.join(root, file)
                    zipf.write(full_path, os.path.relpath(full_path, "/kaggle/working"))

print(f"✅ All results archived in {ZIP_NAME}")

✅ All results archived in Dental_Research_Export.zip
